In [1]:
from abides_core import abides
from abides_core.agent import Agent
from abides_core.communicative_agent import CommunicativeAgent
from abides_core.network import *
from abides_core.message import Message

In [2]:
from itertools import count

id_gen = count(0)

# Message Processing Models

In [3]:
from abides_markets.messages.trading_signal import SellSignal, BuySignal
from abides_core.message_processing_model import MostRecentProcessingModel, VotingProcessingModel


class ExampleAgent3(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(BuySignal(symbol="test"))
        self.set_wakeup(current_time + 20)


class ExampleAgent4(CommunicativeAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []

    def wakeup(self, current_time):
        super().wakeup(current_time)
        self.broadcast(SellSignal(symbol="test"))
        self.set_wakeup(current_time + 20)


class ExampleAgent5(CommunicativeAgent):
    def __init__(self, processing_model, **kwargs):
        super().__init__(**kwargs)
        self.received_messages = []
        self.message_processing_model = processing_model
        self.final_decision = None

    def wakeup(self, current_time):
        super().wakeup(current_time)

    def receive_message(self, current_time, sender_id, message):
        super().receive_message(current_time, sender_id, message)
        self.message_processing_model.on_receive(current_time, message, sender_id)
        self.received_messages.append((current_time, sender_id, message))

    def kernel_terminating(self):
        self.final_decision = type(self.message_processing_model.on_decision())
        super().kernel_terminating()


In [4]:
# test 0: dwa centra jedno most recent, jedno voting
# test probabilistic: dwa centra jedno TrustWeighted, jedno TrustedSource, agenci wysyłający sygnał raz na x nanosekund

In [4]:
agents_graph_1 = [ExampleAgent4(id=next(id_gen)) for i in range(4)]
agents_graph_1.extend([ExampleAgent3(id=next(id_gen)) for i in range(6)])

agents_graph_2 = [ExampleAgent3(id=next(id_gen)) for i in range(5)]
agents_graph_2.extend([ExampleAgent4(id=next(id_gen)) for i in range(5)])

central_agent_1 = ExampleAgent5(processing_model=VotingProcessingModel(), id=next(id_gen))
central_agent_2 = ExampleAgent5(processing_model=MostRecentProcessingModel(), id=next(id_gen))

graph_1 = CentralizedNetwork.construct_from_agent_list(central_agent=central_agent_1, agent_list=agents_graph_1)
graph_2 = CentralizedNetwork.construct_from_agent_list(central_agent=central_agent_2, agent_list=agents_graph_2)

In [5]:
central_agent_1.id

20

In [6]:
sim_agents = graph_1.get_agent_list()
sim_agents.extend(graph_2.get_agent_list())
sim_agents.sort(key=lambda agent: agent.id)

In [7]:
[sim_agent.id for sim_agent in iter(sim_agents)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [8]:
end_state = abides.run(config={"agents": sim_agents, "stdout_log_level": "INFO"})

[5476] INFO abides Simulation Start Time: 2023-05-23 18:56:15.152095
[5476] INFO abides_core.kernel --- Simulation time: 1970-01-01 09:30:00, messages processed: 0, wallclock elapsed: 0.00s ---
[5476] INFO abides_core.kernel Event Queue elapsed: 0:00:00.007000, messages: 152, messages per second: 21714.3
[5476] INFO abides_core.kernel Mean ending value by agent type:
[5476] INFO abides_core.kernel Simulation ending!
[5476] INFO abides Simulation End Time: 2023-05-23 18:56:15.197098
[5476] INFO abides Time taken to run simulation: 0:00:00.045003


In [9]:
end_state['agents']

In [10]:
# czemu nonetype? czy dostaje właściwą wiadomosc?
end_state['agents'][20].received_messages

[(34200000000002, 0, SellSignal(message_id=23, symbol='test')),
 (34200000000003, 1, SellSignal(message_id=24, symbol='test')),
 (34200000000004, 2, SellSignal(message_id=25, symbol='test')),
 (34200000000005, 3, SellSignal(message_id=26, symbol='test')),
 (34200000000006, 4, BuySignal(message_id=27, symbol='test')),
 (34200000000007, 5, BuySignal(message_id=28, symbol='test')),
 (34200000000008, 6, BuySignal(message_id=29, symbol='test')),
 (34200000000009, 7, BuySignal(message_id=30, symbol='test')),
 (34200000000010, 8, BuySignal(message_id=31, symbol='test')),
 (34200000000011, 9, BuySignal(message_id=32, symbol='test'))]

In [11]:
end_state['agents'][21].final_decision

abides_markets.messages.trading_signal.SellSignal

In [12]:
end_state['agents'][20].final_decision

abides_markets.messages.trading_signal.SellSignal

# *Friend*, *FalseFriend*, *Insider*

In [31]:
from abides_markets.agents.network_agents import Friend, FalseFriend

graph_1.get_agent_list()